# Capstone Project - Coursera Data Science

I want to buy a house or an apartment in the greater Munich area. Munich is quite expensive; therefore, I want to know, which neighborhoods and cities are similar to each other

Library installation

In [2]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

print('Libraries imported.')

Solving environment: done

## Package Plan ##

  environment location: /home/jupyterlab/conda/envs/python

  added / updated specs:
    - geopy


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    geographiclib-1.50         |             py_0          34 KB  conda-forge
    geopy-1.21.0               |             py_0          58 KB  conda-forge
    ------------------------------------------------------------
                                           Total:          92 KB

The following NEW packages will be INSTALLED:

  geographiclib      conda-forge/noarch::geographiclib-1.50-py_0
  geopy              conda-forge/noarch::geopy-1.21.0-py_0



geopy-1.21.0         | 58 KB     | ##################################### | 100% 
geographiclib-1.50   | 34 KB     | ##################################### | 100% 
Preparing transaction: done
Verifying transaction: done
Executing transaction: done
Sol

## Open Geojson Data from Bavaria to get coordinates

In [3]:
with open('greatermunich.geojson') as json_data:
    greater_munich_geo = json.load(json_data)
#with open('munich.geojson') as json_data:
#    munich_geo = json.load(json_data)
    
greater_munich_data=greater_munich_geo["features"]
#munich_data=munich_geo["features"]

In [4]:
column_names = ['name', 'coordinates', 'center_latitude', 'center_longitude','plz'] #create new dataframe for geo data
neighborhoods = pd.DataFrame(columns=column_names)
neighborhoods

,name,coordinates,center_latitude,center_longitude,plz


In [5]:
greater_munich_data[0] #have a look at the data and fine the coordinates

{'type': 'Feature',
 'properties': {'ADE': 6,
  'GF': 4,
  'BSG': 1,
  'RS': '091740111111',
  'AGS': '09174111',
  'SDV_RS': '091740111111',
  'GEN': 'Altomünster',
  'BEZ': 'Gemeinde',
  'IBZ': 62,
  'BEM': '--',
  'NBD': 'ja',
  'SN_L': '09',
  'SN_R': '1',
  'SN_K': '74',
  'SN_V1': '01',
  'SN_V2': '11',
  'SN_G': '111',
  'FK_S3': 'R',
  'NUTS': 'DE217',
  'RS_0': '091740111111',
  'AGS_0': '09174111',
  'WSK': '2009/01/01',
  'DEBKG_ID': 'DEBKGDL20000DY9S',
  'destatis': {'RS': '091740111111',
   'area': 75.66,
   'population': 7925,
   'population_m': 3994,
   'population_w': 3931,
   'population_density': 105,
   'zip': '85250',
   'center_lon': '11,256047',
   'center_lat': '48,387627',
   'travel_key': 'T24',
   'travel_desc': 'Münchener Umland',
   'density_key': '03',
   'density_desc': 'gering besiedelt'},
  'wikipedia': {'cid': 'http://www.wikidata.org/entity/Q443815',
   'name': 'Altomünster',
   'AGS': '09174111',
   '_licence_plate_code': 'DAH',
   '_official_website'

In [6]:
for data in greater_munich_data:  #put coordinates in dataframe
    name = data["properties"]['GEN']  
    neighborhood_latlon = data['geometry']['coordinates']
    try:
        long= data["properties"]['destatis']["center_lon"]
        lat = data["properties"]['destatis']["center_lat"]
        plz =data["properties"]['destatis']["zip"]
    except:
        long="NaN"
        lat="NaN"
    
    neighborhoods = neighborhoods.append({'name': name,
                                         'coordinates': neighborhood_latlon, "center_latitude":lat, "center_longitude":long, "plz":plz}, ignore_index=True)
print(neighborhoods.shape)
neighborhoods.head() #have a look

(112, 5)


,name,coordinates,center_latitude,center_longitude,plz
0,Altomünster,"[[[11.255496723984578, 48.44100240625244], [11...","48,387627","11,256047",85250
1,Bergkirchen,"[[[11.37609221393309, 48.29650120908578], [11....","48,259031","11,366061",85232
2,Dachau,"[[[11.409027950554307, 48.30988065943525], [11...","48,263327","11,437493",85221
3,Erdweg,"[[[11.298983299739454, 48.37375152156056], [11...","48,332399","11,303171",85253
4,Haimhausen,"[[[11.522224823170538, 48.351974267420346], [1...","48,3162","11,553037",85778


In [7]:
munich_west= pd.read_csv("munich_west.csv", sep=";") #import Munich neighborhoods and have a look
munich_west

,name,center_longitude,center_latitude
0,Sendling-Westpark,11.531944,48.122222
1,"Thalkirchen, Obersendling",11.519444,48.088889
2,Hadern,11.465278,48.110556
3,Pasing,11.459722,48.145833
4,Aubing-Lochhausen-Langwied,11.419444,48.158333


In [8]:
neighborhoods = neighborhoods.append(munich_west)
neighborhoods.tail()

,name,coordinates,center_latitude,center_longitude,plz
0,Sendling-Westpark,NaN,48.1222,11.5319,NaN
1,"Thalkirchen, Obersendling",NaN,48.0889,11.5194,NaN
2,Hadern,NaN,48.1106,11.4653,NaN
3,Pasing,NaN,48.1458,11.4597,NaN
4,Aubing-Lochhausen-Langwied,NaN,48.1583,11.4194,NaN


In [9]:
neighborhoods.head()

,name,coordinates,center_latitude,center_longitude,plz
0,Altomünster,"[[[11.255496723984578, 48.44100240625244], [11...","48,387627","11,256047",85250
1,Bergkirchen,"[[[11.37609221393309, 48.29650120908578], [11....","48,259031","11,366061",85232
2,Dachau,"[[[11.409027950554307, 48.30988065943525], [11...","48,263327","11,437493",85221
3,Erdweg,"[[[11.298983299739454, 48.37375152156056], [11...","48,332399","11,303171",85253
4,Haimhausen,"[[[11.522224823170538, 48.351974267420346], [1...","48,3162","11,553037",85778


German coordinates have to be converted in an English format

In [10]:
neighborhoods['center_latitude'] = neighborhoods['center_latitude'].astype(str)
neighborhoods['center_longitude'] = neighborhoods['center_longitude'].astype(str)
neighborhoods['center_latitude'] = neighborhoods['center_latitude'].str.replace(',','.')
neighborhoods['center_longitude'] = neighborhoods['center_longitude'].str.replace(',','.')
neighborhoods.head() 

,name,coordinates,center_latitude,center_longitude,plz
0,Altomünster,"[[[11.255496723984578, 48.44100240625244], [11...",48.387627,11.256047,85250
1,Bergkirchen,"[[[11.37609221393309, 48.29650120908578], [11....",48.259031,11.366061,85232
2,Dachau,"[[[11.409027950554307, 48.30988065943525], [11...",48.263327,11.437493,85221
3,Erdweg,"[[[11.298983299739454, 48.37375152156056], [11...",48.332399,11.303171,85253
4,Haimhausen,"[[[11.522224823170538, 48.351974267420346], [1...",48.3162,11.553037,85778


In [11]:
neighborhoods.tail()

,name,coordinates,center_latitude,center_longitude,plz
0,Sendling-Westpark,NaN,48.122222,11.531944000000001,NaN
1,"Thalkirchen, Obersendling",NaN,48.088889,11.519444,NaN
2,Hadern,NaN,48.110556,11.465278,NaN
3,Pasing,NaN,48.145833,11.459722,NaN
4,Aubing-Lochhausen-Langwied,NaN,48.158333,11.419444,NaN


Everything looks fine

In [12]:
print(neighborhoods['center_longitude'].dtypes) #convert into float, that folium can read it
neighborhoods['center_latitude'] = neighborhoods['center_latitude'].astype(float) 
neighborhoods['center_longitude'] = neighborhoods['center_longitude'].astype(float) 
neighborhoods['center_longitude'].dtypes

object


dtype('float64')

In [13]:
neighborhoods.head()

,name,coordinates,center_latitude,center_longitude,plz
0,Altomünster,"[[[11.255496723984578, 48.44100240625244], [11...",48.387627,11.256047,85250
1,Bergkirchen,"[[[11.37609221393309, 48.29650120908578], [11....",48.259031,11.366061,85232
2,Dachau,"[[[11.409027950554307, 48.30988065943525], [11...",48.263327,11.437493,85221
3,Erdweg,"[[[11.298983299739454, 48.37375152156056], [11...",48.332399,11.303171,85253
4,Haimhausen,"[[[11.522224823170538, 48.351974267420346], [1...",48.316200,11.553037,85778


In [14]:
neighborhoods.dropna(subset = ["center_latitude"], inplace=True) #drop NaN, because folium can't deal with NaNs
neighborhoods.dropna(subset = ["center_longitude"], inplace=True)

neighborhoods["name"] #have a look at the cities and neighborhoods

0                     Altomünster
1                     Bergkirchen
2                          Dachau
3                          Erdweg
4                      Haimhausen
5                   Hebertshausen
6                       Karlsfeld
7                Markt Indersdorf
8                     Odelzhausen
9                    Petershausen
10        Pfaffenhofen a.d. Glonn
11                       Röhrmoos
12                   Schwabhausen
13                      Sulzemoos
14         Hilgertshausen-Tandern
15                    Vierkirchen
16                         Weichs
17                         Alling
18                      Egenhofen
19                       Eichenau
20                       Emmering
21               Fürstenfeldbruck
22                      Germering
23                     Gröbenzell
24                        Maisach
25                     Moorenweis
26                        Olching
27                       Puchheim
28                     Türkenfeld
29            

In [15]:
neighborhoods=neighborhoods.reset_index() #reset index
neighborhoods["name"]

0                     Altomünster
1                     Bergkirchen
2                          Dachau
3                          Erdweg
4                      Haimhausen
5                   Hebertshausen
6                       Karlsfeld
7                Markt Indersdorf
8                     Odelzhausen
9                    Petershausen
10        Pfaffenhofen a.d. Glonn
11                       Röhrmoos
12                   Schwabhausen
13                      Sulzemoos
14         Hilgertshausen-Tandern
15                    Vierkirchen
16                         Weichs
17                         Alling
18                      Egenhofen
19                       Eichenau
20                       Emmering
21               Fürstenfeldbruck
22                      Germering
23                     Gröbenzell
24                        Maisach
25                     Moorenweis
26                        Olching
27                       Puchheim
28                     Türkenfeld
29            

In [16]:
#neighborhoods[neighborhoods["name"]=="Aubing-Lochhausen-Langwied"]
neighborhoods.tail()
del neighborhoods['index']

In [17]:

neighborhoods.tail()

,name,coordinates,center_latitude,center_longitude,plz
108,Sendling-Westpark,NaN,48.122222,11.531944,NaN
109,"Thalkirchen, Obersendling",NaN,48.088889,11.519444,NaN
110,Hadern,NaN,48.110556,11.465278,NaN
111,Pasing,NaN,48.145833,11.459722,NaN
112,Aubing-Lochhausen-Langwied,NaN,48.158333,11.419444,NaN


In [18]:
map_bavaria = folium.Map(location=[48.137222, 11.575556], zoom_start=9) #create a map to show the loci

# add markers to map
for lat, lng, name in zip(neighborhoods['center_latitude'], neighborhoods['center_longitude'], neighborhoods['name']):
    #print(name)
    label = '{}'.format(name)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_bavaria)  
    
map_bavaria

In [19]:
CLIENT_ID = 'M2BHNOKFP4AET41J0SBITQOTEH3GM0WRZL3EAOSPBIEWIQDM' # your Foursquare ID #connect to forsquare
CLIENT_SECRET = 'ZWRL2Y0L0EWJTQRRFQMT1GA0GNV2V4CSBC1CTRQORBB5XPQY' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: M2BHNOKFP4AET41J0SBITQOTEH3GM0WRZL3EAOSPBIEWIQDM
CLIENT_SECRET:ZWRL2Y0L0EWJTQRRFQMT1GA0GNV2V4CSBC1CTRQORBB5XPQY


In [20]:
# function that extracts the category of the venue 
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [21]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [22]:
LIMIT = 100 # limit of number of venues returned by Foursquare API
bavaria_venues = getNearbyVenues(names=neighborhoods['name'],
                                   latitudes=neighborhoods['center_latitude'],
                                   longitudes=neighborhoods['center_longitude']
                                  )


Altomünster
Bergkirchen
Dachau
Erdweg
Haimhausen
Hebertshausen
Karlsfeld
Markt Indersdorf
Odelzhausen
Petershausen
Pfaffenhofen a.d. Glonn
Röhrmoos
Schwabhausen
Sulzemoos
Hilgertshausen-Tandern
Vierkirchen
Weichs
Alling
Egenhofen
Eichenau
Emmering
Fürstenfeldbruck
Germering
Gröbenzell
Maisach
Moorenweis
Olching
Puchheim
Türkenfeld
Adelshofen
Althegnenberg
Hattenhofen
Jesenwang
Landsberied
Mammendorf
Mittelstetten
Oberschweinbach
Grafrath
Kottgeisering
Schöngeising
Aschheim
Baierbrunn
Oberhaching
Oberschleißheim
Ottobrunn
Aying
Planegg
Pullach i. Isartal
Putzbrunn
Sauerlach
Schäftlarn
Straßlach-Dingharting
Taufkirchen
Neubiberg
Brunnthal
Feldkirchen
Garching b. München
Gräfelfing
Grasbrunn
Grünwald
Haar
Höhenkirchen-Siegertsbrunn
Hohenbrunn
Ismaning
Kirchheim b. München
Neuried
Unterföhring
Unterhaching
Unterschleißheim
Berg
Andechs
Feldafing
Gauting
Gilching
Herrsching a. Ammersee
Inning a. Ammersee
Krailling
Seefeld
Pöcking
Starnberg
Tutzing
Weßling
Wörthsee
Kissing
Merching
Rehling
R

In [23]:
print(bavaria_venues.shape)
bavaria_venues.head()

(676, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Altomünster,48.387627,11.256047,Kapplerbräu,48.387158,11.257675,German Restaurant
1,Altomünster,48.387627,11.256047,EDEKA,48.387393,11.253661,Supermarket
2,Altomünster,48.387627,11.256047,Brauereigasthof Maierbräu,48.387684,11.256150,Hotel
3,Altomünster,48.387627,11.256047,S Altomünster,48.385314,11.254667,Train Station
4,Bergkirchen,48.259031,11.366061,Stern Apotheke,48.257834,11.366005,Pharmacy


In [24]:
bavaria_venues.groupby('Neighborhood').count() #group the neighborhoods

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Adelshofen,1,1,1,1,1,1
Adelzhausen,2,2,2,2,2,2
Affing,2,2,2,2,2,2
Aichach,7,7,7,7,7,7
Aindling,3,3,3,3,3,3
Alling,5,5,5,5,5,5
Althegnenberg,3,3,3,3,3,3
Altomünster,4,4,4,4,4,4
Andechs,5,5,5,5,5,5


In [25]:
print('There are {} uniques categories.'.format(len(bavaria_venues['Venue Category'].unique())))

There are 151 uniques categories.


In [26]:
# one hot encoding
bavaria_onehot = pd.get_dummies(bavaria_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
bavaria_onehot['Neighborhood'] = bavaria_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [bavaria_onehot.columns[-1]] + list(bavaria_onehot.columns[:-1])
bavaria_onehot = bavaria_onehot[fixed_columns]

bavaria_onehot.head()

,Neighborhood,Asian Restaurant,Athletics & Sports,Auto Dealership,Auto Garage,Bakery,Bank,Bar,Baseball Field,Bavarian Restaurant,Beach,Beach Bar,Beer Garden,Board Shop,Boarding House,Bookstore,Boutique,Breakfast Spot,Brewery,Buffet,Burger Joint,Bus Station,Bus Stop,Business Service,Butcher,Cafeteria,Café,Casino,Cheese Shop,Chinese Restaurant,Clothing Store,Cocktail Bar,Coffee Roaster,Coffee Shop,Community Center,Concert Hall,Construction & Landscaping,Cosmetics Shop,Cupcake Shop,Dance Studio,Deli / Bodega,Dentist's Office,Dessert Shop,Discount Store,Doctor's Office,Dog Run,Donut Shop,Drugstore,Eastern European Restaurant,Electronics Store,Event Service,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Fish Market,Food & Drink Shop,Food Court,Forest,Fried Chicken Joint,Furniture / Home Store,Garden,Garden Center,Gas Station,Gastropub,German Restaurant,Gift Shop,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Gym Pool,Health & Beauty Service,Historic Site,Hotel,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Inn,Insurance Office,Italian Restaurant,Japanese Restaurant,Jewelry Store,Kebab Restaurant,Lake,Laser Tag,Light Rail Station,Liquor Store,Lottery Retailer,Lounge,Market,Mediterranean Restaurant,Mexican Restaurant,Miscellaneous Shop,Mobile Phone Shop,Modern European Restaurant,Mongolian Restaurant,Motel,Motorcycle Shop,Movie Theater,Museum,Nail Salon,Office,Organic Grocery,Palace,Paper / Office Supplies Store,Park,Pastry Shop,Perfume Shop,Pet Store,Pharmacy,Photography Studio,Pier,Pizza Place,Playground,Plaza,Pool,Post Office,Residential Building (Apartment / Condo),Restaurant,River,Roof Deck,Salon / Barbershop,Sandwich Place,Scenic Lookout,Seafood Restaurant,Shipping Store,Shopping Mall,Shopping Plaza,Skate Park,Ski Area,Snack Place,Soccer Field,Soccer Stadium,Sporting Goods Shop,Sports Bar,Steakhouse,Supermarket,Sushi Restaurant,Taverna,Tea Room,Tennis Court,Thai Restaurant,Theater,Theme Park,Theme Restaurant,Toy / Game Store,Train Station,Trattoria/Osteria,Turkish Restaurant,Vietnamese Restaurant,Yoga Studio
0,Altomünster,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,Altomünster,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,Altomünster,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,Altomünster,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0
4,Bergkirchen,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [27]:
bavaria_onehot.shape

(676, 152)

In [28]:
bavaria_grouped = bavaria_onehot.groupby('Neighborhood').mean().reset_index()
bavaria_grouped

,Neighborhood,Asian Restaurant,Athletics & Sports,Auto Dealership,Auto Garage,Bakery,Bank,Bar,Baseball Field,Bavarian Restaurant,Beach,Beach Bar,Beer Garden,Board Shop,Boarding House,Bookstore,Boutique,Breakfast Spot,Brewery,Buffet,Burger Joint,Bus Station,Bus Stop,Business Service,Butcher,Cafeteria,Café,Casino,Cheese Shop,Chinese Restaurant,Clothing Store,Cocktail Bar,Coffee Roaster,Coffee Shop,Community Center,Concert Hall,Construction & Landscaping,Cosmetics Shop,Cupcake Shop,Dance Studio,Deli / Bodega,Dentist's Office,Dessert Shop,Discount Store,Doctor's Office,Dog Run,Donut Shop,Drugstore,Eastern European Restaurant,Electronics Store,Event Service,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Fish Market,Food & Drink Shop,Food Court,Forest,Fried Chicken Joint,Furniture / Home Store,Garden,Garden Center,Gas Station,Gastropub,German Restaurant,Gift Shop,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Gym Pool,Health & Beauty Service,Historic Site,Hotel,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Inn,Insurance Office,Italian Restaurant,Japanese Restaurant,Jewelry Store,Kebab Restaurant,Lake,Laser Tag,Light Rail Station,Liquor Store,Lottery Retailer,Lounge,Market,Mediterranean Restaurant,Mexican Restaurant,Miscellaneous Shop,Mobile Phone Shop,Modern European Restaurant,Mongolian Restaurant,Motel,Motorcycle Shop,Movie Theater,Museum,Nail Salon,Office,Organic Grocery,Palace,Paper / Office Supplies Store,Park,Pastry Shop,Perfume Shop,Pet Store,Pharmacy,Photography Studio,Pier,Pizza Place,Playground,Plaza,Pool,Post Office,Residential Building (Apartment / Condo),Restaurant,River,Roof Deck,Salon / Barbershop,Sandwich Place,Scenic Lookout,Seafood Restaurant,Shipping Store,Shopping Mall,Shopping Plaza,Skate Park,Ski Area,Snack Place,Soccer Field,Soccer Stadium,Sporting Goods Shop,Sports Bar,Steakhouse,Supermarket,Sushi Restaurant,Taverna,Tea Room,Tennis Court,Thai Restaurant,Theater,Theme Park,Theme Restaurant,Toy / Game Store,Train Station,Trattoria/Osteria,Turkish Restaurant,Vietnamese Restaurant,Yoga Studio
0,Adelshofen,0.000000,0.00,0.00,0.0,0.000000,0.000000,0.000000,0.000,0.000000,0.00,0.000000,0.000000,0.0,0.00,0.000000,0.000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.0,0.000000,0.00,0.000000,0.00,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.0,0.00,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.00,1.000000,0.00,0.00,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00
1,Adelzhausen,0.000000,0.00,0.00,0.0,0.000000,0.000000,0.000000,0.000,0.000000,0.00,0.000000,0.000000,0.0,0.00,0.000000,0.000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.0,0.000000,0.00,0.000000,0.00,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.0,0.00,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.00,0.000000,0.00,0.00,0.000000,0.000000,0.00,0.000000,0.500000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.0

In [29]:
bavaria_grouped.shape

(110, 152)

In [30]:
num_top_venues = 5 #show venues

for hood in bavaria_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = bavaria_grouped[bavaria_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Adelshofen----
              venue  freq
0  Greek Restaurant   1.0
1  Asian Restaurant   0.0
2   Motorcycle Shop   0.0
3     Movie Theater   0.0
4            Museum   0.0


----Adelzhausen----
                venue  freq
0               Hotel   0.5
1  Photography Studio   0.5
2    Asian Restaurant   0.0
3              Palace   0.0
4     Motorcycle Shop   0.0


----Affing----
                  venue  freq
0           Supermarket   0.5
1             Bookstore   0.5
2      Asian Restaurant   0.0
3  Mongolian Restaurant   0.0
4       Motorcycle Shop   0.0


----Aichach----
               venue  freq
0               Café  0.29
1  Food & Drink Shop  0.14
2     Ice Cream Shop  0.14
3              Hotel  0.14
4         Steakhouse  0.14


----Aindling----
              venue  freq
0       Supermarket  0.33
1          Gym Pool  0.33
2        Nail Salon  0.33
3  Asian Restaurant  0.00
4   Organic Grocery  0.00


----Alling----
                    venue  freq
0                     Gym   0.2
1 

In [31]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [32]:
num_top_venues = 10 #most common venues

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = bavaria_grouped['Neighborhood']

for ind in np.arange(bavaria_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(bavaria_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Adelshofen,Greek Restaurant,Yoga Studio,Drugstore,Fish Market,Fast Food Restaurant,Farmers Market,Falafel Restaurant,Event Service,Electronics Store,Eastern European Restaurant
1,Adelzhausen,Hotel,Photography Studio,Drugstore,Fish Market,Fast Food Restaurant,Farmers Market,Falafel Restaurant,Event Service,Electronics Store,Eastern European Restaurant
2,Affing,Bookstore,Supermarket,Yoga Studio,Food Court,Fish Market,Fast Food Restaurant,Farmers Market,Falafel Restaurant,Event Service,Electronics Store
3,Aichach,Café,Ice Cream Shop,Steakhouse,Lottery Retailer,Food & Drink Shop,Hotel,Discount Store,Doctor's Office,Dog Run,Donut Shop
4,Aindling,Gym Pool,Supermarket,Nail Salon,Yoga Studio,Eastern European Restaurant,Fast Food Restaurant,Farmers Market,Falafel Restaurant,Event Service,Electronics Store


In [33]:
# set number of clusters
kclusters = 5

bavaria_grouped_clustering = bavaria_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(bavaria_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([2, 4, 0, 2, 0, 2, 0, 4, 4, 4], dtype=int32)

In [35]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)
print(neighborhoods_venues_sorted.head())
bavaria_merged = neighborhoods

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
bavaria_merged = bavaria_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='name')

bavaria_merged.head() # check the last columns!

   Cluster Labels Neighborhood 1st Most Common Venue 2nd Most Common Venue  \
0               2   Adelshofen      Greek Restaurant           Yoga Studio   
1               4  Adelzhausen                 Hotel    Photography Studio   
2               0       Affing             Bookstore           Supermarket   
3               2      Aichach                  Café        Ice Cream Shop   
4               0     Aindling              Gym Pool           Supermarket   

  3rd Most Common Venue 4th Most Common Venue        5th Most Common Venue  \
0             Drugstore           Fish Market         Fast Food Restaurant   
1             Drugstore           Fish Market         Fast Food Restaurant   
2           Yoga Studio            Food Court                  Fish Market   
3            Steakhouse      Lottery Retailer            Food & Drink Shop   
4            Nail Salon           Yoga Studio  Eastern European Restaurant   

  6th Most Common Venue 7th Most Common Venue 8th Most Common 

,name,coordinates,center_latitude,center_longitude,plz,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Altomünster,"[[[11.255496723984578, 48.44100240625244], [11...",48.387627,11.256047,85250,4.0,Hotel,German Restaurant,Train Station,Supermarket,Fast Food Restaurant,Farmers Market,Falafel Restaurant,Event Service,Electronics Store,Eastern European Restaurant
1,Bergkirchen,"[[[11.37609221393309, 48.29650120908578], [11....",48.259031,11.366061,85232,2.0,Pharmacy,Bank,German Restaurant,Yoga Studio,Eastern European Restaurant,Fish Market,Fast Food Restaurant,Farmers Market,Falafel Restaurant,Event Service
2,Dachau,"[[[11.409027950554307, 48.30988065943525], [11...",48.263327,11.437493,85221,2.0,Park,Greek Restaurant,German Restaurant,Bar,Boutique,Lottery Retailer,Eastern European Restaurant,Fast Food Restaurant,Farmers Market,Falafel Restaurant
3,Erdweg,"[[[11.298983299739454, 48.37375152156056], [11...",48.332399,11.303171,85253,0.0,German Restaurant,Athletics & Sports,Train Station,Bank,Supermarket,Yoga Studio,Fast Food Restaurant,Farmers Market,Falafel Restaurant,Event Service
4,Haimhausen,"[[[11.522224823170538, 48.351974267420346], [1...",48.316200,11.553037,85778,0.0,German Restaurant,Supermarket,Restaurant,Casino,Drugstore,Farmers Market,Falafel Restaurant,Event Service,Electronics Store,Eastern European Restaurant


In [36]:
bavaria_merged['Cluster Labels']=bavaria_merged['Cluster Labels'].fillna(0)
# create map
map_clusters = folium.Map(location=[48.137222, 11.575556], zoom_start=9)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(bavaria_merged['center_latitude'], bavaria_merged['center_longitude'], bavaria_merged['name'], bavaria_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[int(cluster-1)],
        fill=True,
        fill_color=rainbow[int(cluster-1)],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters